<a href="https://colab.research.google.com/github/cristianmejia00/clustering/blob/main/Topic_Models_using_BERTopic_TOPIC_MODEL_20241101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling with BERTopic

🔴 copied from the [Kubota Colab](https://colab.research.google.com/drive/1YsDp5_qGXGJKsEXsS8DO8CA_lqZc6EpA).  

`Topic Models` are methods to automatically organize a corpus of text into topics.

Topic Model process:
1. Data preparation
2. Tranform text to numeric vectors
3. Multidimensionality reduction
4. Clustering
5. Topic analysis
6. Cluster assignation


This notebook uses the library `BERTopic` which is a one-stop solution for topic modeling including handy functions for plotting and analysis. However, BERTopic does not have a function to extract the X and Y coords from UMAP. If we need the coordinates then use the notebooks `Topic_Models_using_Transformers` instead. In any other situation, when a quick analysis is needed this notebook may be better.

This notebook is also the one needed for the heatmap codes included in this folder.

`BERTopic` is Python library that handles steps 2 to 6.
BERT topic models use the transformer architechture to generate the embeds (i.e. the vector or numeric representation of words) and are currently the state-of-the-art method for vectorization.

This notebook shows how to use it.

---
Reading:
[Topic Modeling with Deep Learning Using Python BERTopic](https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504)
[Advanced Topic Modeling with BERTopic](https://www.pinecone.io/learn/bertopic/)


# Requirements

## Packages installation and initialization

In [2]:
!pip install bertopic[visualization]

zsh:1: no matches found: bertopic[visualization]


In [1]:
import pandas as pd
import numpy as np
import time
import math
import uuid
import re
import os
import json
import pickle
from datetime import date
from itertools import compress
from bertopic import BERTopic
from umap import UMAP
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

/Users/cristian/Desktop/GitHub/clustering/env-tm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect your Google Drive

In [64]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
def find_e_keys(dictionary):
    # List comprehension to find keys starting with 'e'
    e_keys = [key for key in dictionary if str(key).lower().startswith('e')]
    return e_keys

# 🔴 Input files and options

Go to your Google Drive and create a folder in the root directory. We are going to save all related data in that directory.
Upload the dataset of news into the above folder.
- The dataset should be a `.csv` file.
- Every row in the dataset is a document
- It can any kind of columns. Some columns must contain the text we want to analyze. For example, a dataset of academic articles may contain a "Title" and/or "Abstract" column.

In [4]:
# The bibliometrics folder
# Colab
ROOT_FOLDER_PATH = "drive/MyDrive/Bibliometrics_Drive"

# Mac
ROOT_FOLDER_PATH = "/Users/cristian/Library/CloudStorage/GoogleDrive-cristianmejia00@gmail.com/My Drive/Bibliometrics_Drive"

# Change to the name of the folder where the dataset is uploaded inside the above folder
project_folder = 'Q325_ai_libsci'

analysis_id = 'a01_tm__f01_e01__km01'

# Filtered label
settings_directive = "settings_analysis_directive_2025-01-30-23-47.json"

In [5]:
# Read settings
with open(f'{ROOT_FOLDER_PATH}/{project_folder}/{analysis_id}/{settings_directive}', 'r') as file:
    settings = json.load(file)

In [6]:
# Input dataset
dataset_file_path = f"{ROOT_FOLDER_PATH}/{settings['metadata']['project_folder']}/{settings['metadata']['filtered_folder']}/dataset_raw_cleaned.csv"

In [7]:
# Function to save files
def save_as_csv(df, save_name_without_extension, with_index):
    "usage: `save_as_csv(dataframe, 'filename')`"
    df.to_csv(f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv", index=with_index)
    print("===\nSaved: ", f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv")

In [8]:
# prompt: a function to save object to a pickle file
def save_object_as_pickle(obj, filename):
  """
  Saves an object as a pickle file.

  Args:
      obj: The object to be saved.
      filename: The filename of the pickle file.
  """
  with open(filename, "wb") as f:
    pickle.dump(obj, f)


In [9]:
# prompt: a function to load pickle object given a path
def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [10]:
# Open the data file
df = pd.read_csv(f"{dataset_file_path}", encoding='latin-1')
print(df.shape)
df.head()

(2795, 42)


,X_N,uuid,PT,AU,AF,TI,SO,LA,DT,DE,...,AR,DI,PG,WC,SC,OA,UT,Countries,IsoCountries,Institutions
0,1,06ffaeca-2840-45f5-97d7-3adb96371bf8,J,"de Leon, LCR; Flores, LV; Alomo, ARL","de Leon, Lady Catherine R.; Flores, Lejempf V....",Artificial Intelligence and Filipino Academic ...,JOURNAL OF THE AUSTRALIAN LIBRARY AND INFORMAT...,English,Article,Filipino librarians; academic librarians; arti...,...,NaN,10.1080/24750158.2024.2305993,18,Information Science & Library Science,Information Science & Library Science,NaN,WOS:001153914800001,philippines,PHL,univ santo tomas
1,2,151cecd6-3d80-4ffa-835f-b894ff2a97b1,J,"Xu, M; Liu, DA; Zhang, Y","Xu, Min; Liu, DongAo; Zhang, Yan",Design of Interactive Teaching System of Physi...,JOURNAL OF INFORMATION & KNOWLEDGE MANAGEMENT,English,Article,Artificial intelligence; physical training; in...,...,2240021,10.1142/S0219649222400214,16,Information Science & Library Science,Information Science & Library Science,NaN,WOS:000821701500002,peoples r china,CHN,shanghai univ finance & econ shanghai
2,3,70ea7e13-c197-466b-a5ad-80d350d2d431,J,"Filson, CK; Atuase, D","Filson, Christopher K.; Atuase, Diana",Artificial intelligence and academic integrity...,INFORMATION DEVELOPMENT,English,Article; Early Access,artificial intelligence; academic integrity; i...,...,NaN,10.1177/02666669241284230,13,Information Science & Library Science,Information Science & Library Science,NaN,WOS:001327261700001,ghana,GHA,univ cape coast
3,4,bca01a9b-ef10-49ad-bf30-c1d0ae79c348,J,"Walter, L; Denter, NM; Kebel, J","Walter, Lothar; Denter, Nils M.; Kebel, Jan",A review on digitalization trends in patent in...,WORLD PATENT INFORMATION,English,Review,Digitalization; Patent search and analysis; Cl...,...,102107,10.1016/j.wpi.2022.102107,11,Information Science & Library Science,Information Science & Library Science,NaN,WOS:000788120900001,germany,DEU,univ bremen
4,5,8fc18d16-8209-4dc4-a16a-05a790ac203a,J,"Borges, AFS; Laurindo, FJB; SpÃ­nola, MM; GonÃ...","Borges, Aline F. S.; Laurindo, Fernando J. B.;...",The strategic use of artificial intelligence i...,INTERNATIONAL JOURNAL OF INFORMATION MANAGEMENT,English,Review,Artificial intelligence; Deep learning; Machin...,...,102225,10.1016/j.ijinfomgt.2020.102225,16,Information Science & Library Science,Information Science & Library Science,NaN,WOS:000618806300006,brazil,BRA,univ sao paulo; univ paulista; fundacao educ i...




---



## PART 2: Topic Model

In [11]:
# bibliometrics_folder
# project_folder
# project_name_suffix
# ROOT_FOLDER_PATH = f"drive/MyDrive/{bibliometrics_folder}"

#############################################################
# Embeddings folder
embeddings_folder_name = settings['tmo']['embeds_folder']

# Which column has the year of the documents?
my_year = settings['tmo']['year_column']

# Number of topics. Select the number of topics to extract.
# Choose 0, for automatic detection.
n_topics = settings['tmo']['n_topics']

# Minimum number of documents per topic
min_topic_size = settings['tmo']['min_topic_size']

In [73]:
# Get the embeddings back.
embeddings = load_pickle(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/embeddings.pck")
corpus =     pd.read_csv(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/corpus.csv").reset_index(drop=True)

In [74]:
# Combine embeddings
documents = corpus.text.to_list()

In [75]:
# corpus['uuid'] = [uuid.uuid4() for _ in range(len(corpus.index))]
# corpus['X_N'] = [i for i in range(1, len(corpus.index)+1)]

In [76]:
len(documents)

2795

In [77]:
#len(embeddings) == len(documents)
len(embeddings['embeddings']) == len(documents)

True

In [78]:
from hdbscan.hdbscan_ import HDBSCAN
# Execute the topic model.
# I suggest changing the values marked with #<---
# The others are the default values and they'll work fine in most cases.
# This will take several minutes to finish.

# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

if n_topics == 0:
  # Initiate topic model with HDBScan (Automatic topic selection)
  topic_model_params = HDBSCAN(min_cluster_size=min_topic_size,
                               metric='euclidean',
                               cluster_selection_method='eom',
                               prediction_data=True)
else:
  # Initiate topic model with K-means (Manual topic selection)
  topic_model_params = KMeans(n_clusters = n_topics)

# Initiate BERTopic
topic_model = BERTopic(umap_model = umap_model,
                       hdbscan_model = topic_model_params,
                       min_topic_size=min_topic_size,
                       #nr_topics=15,          #<--- Footnote 1
                       n_gram_range=(1,3),
                       language='english',
                       calculate_probabilities=True,
                       verbose=True)



# Footnote 1: This controls the number of topics we want AFTER clustering.
# Add a hashtag at the beggining to use the number of topics returned by the topic model.
# When using HDBScan nr_topics will be obtained after orphans removal, and there is no warranty that `nr_topics < HDBScan topics`.
# thus, with HDBScan `nr_topics` means N topics OR LESS.
# When using KMeans nr_topics can be used to further reduce the number of topics.
# We use the topics as returned by the topic model. So we do not need to activate it here.

In [79]:
# Compute topic model
#topics, probabilities = topic_model.fit_transform(documents, embeddings)
topics, probabilities = topic_model.fit_transform(documents, embeddings['embeddings'])

2025-01-30 23:49:28,279 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-30 23:49:34,924 - BERTopic - Dimensionality - Completed ✓
2025-01-30 23:49:34,925 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-30 23:49:35,002 - BERTopic - Cluster - Completed ✓
2025-01-30 23:49:35,007 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-30 23:49:37,578 - BERTopic - Representation - Completed ✓


In [80]:
# Get the list of topics
# Topic = the topic number. From the largest topic.
#         "-1" is the generic topic. Genericr keywords are aggegrated here.
# Count = Documents assigned to this topic
# Name = Top 4 words of the topic based on probability
# Representation = The list of words representing this topic
# Representative_Docs = Documents assigned to this topic
tm_summary = topic_model.get_topic_info()
tm_summary

,Topic,Count,Name,Representation,Representative_Docs
0,0,143,0_libraries_library_the_and,"[libraries, library, the, and, in, of, academi...",[Use of Artificial Intelligence Technologies i...
1,1,120,1_nan_artificial_artificial intelligence_intel...,"[nan, artificial, artificial intelligence, int...","[Demystifying artificial intelligence nan, Enc..."
2,2,113,2_the_of_media_news,"[the, of, media, news, and, in, to, this, that...",[Use of artificial intelligence in synthetic m...
3,3,110,3_and_healthcare_to_of,"[and, healthcare, to, of, the, in, ai, health,...",[A survey of pregnant patients perspectives on...
4,4,110,4_artificialintelligence_nan_nan artificialint...,"[artificialintelligence, nan, nan artificialin...",[ARTIFICIALINTELLIGENCE AND INFORMATIONRETRIEV...
5,5,94,5_the_of_in_and,"[the, of, in, and, research, to, scientific, p...",[Productivity trends and citation impact of di...
6,6,88,6_public_ai_the_government,"[public, ai, the, government, in, of, and, to,...",[Mapping the challenges of Artificial Intellig...
7,7,87,7_and_clinical_to_of,"[and, clinical, to, of, the, for, in, we, with...",[Casebased reasoning using electronic health r...
8,8,82,8_the_and_of_to,"[the, and, of, to, intention, trust, perceived...",[An empirical study of factors influencing usa...
9,9,82,9_the_ai_and_of,"[the, ai, and, of, to, study, adoption, in, pe...",[Role of enterprise information system managem...


In [81]:
# Save the topic model assets
tm_folder_path = f'{ROOT_FOLDER_PATH}/{project_folder}/{settings["metadata"]["analysis_id"]}'

if not os.path.exists(tm_folder_path):
  !mkdir $tm_folder_path

tm_summary.to_csv(f'{tm_folder_path}/topic_model_info.csv', index=False)

In [82]:
# Number of topics found
found_topics = max(tm_summary.Topic) + 1
found_topics

56

In [83]:
# Confirm all documents are assigned
sum(tm_summary.Count) == len(corpus)

True

In [84]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('libraries', 0.0219238562902162),
 ('library', 0.016931208404270922),
 ('the', 0.01333105667948891),
 ('and', 0.012931610618121786),
 ('in', 0.011742400804040693),
 ('of', 0.011518129916442481),
 ('academic', 0.010573681900540142),
 ('ai', 0.010441164644471021),
 ('to', 0.01021897724118123),
 ('librarians', 0.009455601321378335)]

In [85]:
# Get the top 10 documents for a topic
topic_model.get_representative_docs(0)

['Use of Artificial Intelligence Technologies in Rendering Library Services An Empirical Evidence from University Libraries in Africa The main purpose of this paper is to investigate the level of adoption of artificial intelligenceAI to support library services delivery in university libraries in Africa Qualitative research method was adopted to collect data A preliminary survey of  university libraries in English speaking countries in Africa was conducted to identify the university libraries that have adopted AI in their libraries Content analysis was used to analyse the responses The study found that the only few university libraries in Africa have adopted AI technologies such as Chatbot ChatGPT LibKey from ThirdIron robots RFID technology and Grammarly These AI technologies are used to render different library services like answering of directional and ready reference questions posed by library users serving as a knowledge base for cataloguing information of library materials selfch

In [86]:
# Others

# # Get the number of documents per topic (same as in the table above)
# topic_model.get_topic_freq(0)

# # Get the main keywords per topic
# topic_model.get_topics()

In [87]:
# Print the parameters used. (For reporting)
topic_model.get_params()

{'calculate_probabilities': True,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': None,
 'hdbscan_model': KMeans(n_clusters=56),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 3),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_jobs=1, random_state=100, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(ngram_range=(1, 3)),
 'verbose': True,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [88]:
tm_params = dict(topic_model.get_params())
for key, value in tm_params.items():
    tm_params[key]=  str(value)
with open(f'{tm_folder_path}/topic_model_params.json', 'w') as f:
    json.dump(tm_params, f, ensure_ascii=False, indent=4)
    print('Done')

Done


In [89]:
# Get the topic score for each paper and its assigned topic
topic_distr, _ = topic_model.approximate_distribution(documents, batch_size=1000)
distributions = [distr[topic] if topic != -1 else 0 for topic, distr in zip(topics, topic_distr)]

100%|██████████| 3/3 [00:04<00:00,  1.49s/it]


In [90]:
topic_distr

array([[0.50511854, 0.17508381, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.46043877, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.23628946, 0.15901337, 0.03527186, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.0261859 , 0.18536002, 0.0713683 , ..., 0.        , 0.        ,
        0.        ],
       [0.04917815, 0.12997984, 0.0378495 , ..., 0.        , 0.        ,
        0.        ],
       [0.05768826, 0.12999131, 0.0507296 , ..., 0.        , 0.        ,
        0.        ]])

In [91]:
# Document information. Including the topic assignation
dataset_clustering_results = topic_model.get_document_info(documents, df = corpus, metadata={"Score": distributions})

# Standar format for report analysis
dataset_clustering_results = dataset_clustering_results.drop(columns=['text'])
dataset_clustering_results['X_E'] = dataset_clustering_results['Score']
dataset_clustering_results['X_C'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['level0'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['cl99'] = False
dataset_clustering_results['cl-99'] = False
dataset_clustering_results.head()

,UT,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document,Score,X_E,X_C,level0,cl99,cl-99
0,WOS:001153914800001,Artificial Intelligence and Filipino Academic ...,0,0_libraries_library_the_and,"[libraries, library, the, and, in, of, academi...",[Use of Artificial Intelligence Technologies i...,libraries - library - the - and - in - of - ac...,False,0.505119,0.505119,1,1,False,False
1,WOS:000821701500002,Design of Interactive Teaching System of Physi...,13,13_the_of_and_students,"[the, of, and, students, learning, education, ...",[Adaptation algorithms for selecting personali...,the - of - and - students - learning - educati...,False,0.265050,0.265050,14,14,False,False
2,WOS:001327261700001,Artificial intelligence and academic integrity...,0,0_libraries_library_the_and,"[libraries, library, the, and, in, of, academi...",[Use of Artificial Intelligence Technologies i...,libraries - library - the - and - in - of - ac...,False,0.236289,0.236289,1,1,False,False
3,WOS:000788120900001,A review on digitalization trends in patent in...,26,26_patent_the_patents_of,"[patent, the, patents, of, and, in, technology...",[Technological adoptions and sectorspecific in...,patent - the - patents - of - and - in - techn...,False,0.704238,0.704238,27,27,False,False
4,WOS:000618806300006,The strategic use of artificial intelligence i...,9,9_the_ai_and_of,"[the, ai, and, of, to, study, adoption, in, pe...",[Role of enterprise information system managem...,the - ai - and - of - to - study - adoption - ...,False,0.033290,0.033290,10,10,False,False


In [92]:
# Save the dataframe
dataset_clustering_results.to_csv(f'{tm_folder_path}/dataset_minimal.csv', index=False)

In [93]:
# Save the topic model
topic_model.save(f'{tm_folder_path}/topic_model_object.pck')

2025-01-30 23:49:43,725 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.




---

